# Agent

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# Google検索Agent

In [2]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "GoogleSearch", # 名前は半角英数字のみ
        func=search.run,
        description="Search Google for recent results."
    )
]

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2)

In [4]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [5]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [6]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "岸田総理の日米首脳会談の結果について教えてください"})



> Entering new AgentExecutor chain...

Invoking: `GoogleSearch` with `岸田総理の日米首脳会談の結果`


Apr 10, 2024 ... 関連リンク · 日米共同記者会見（官邸HP） · 岸田総理大臣のホワイトハウスでの歓迎式典出席（外務省HP） 別ウィンドウで開く · 日米首脳会談（外務省 ... Nov 16, 2023 ... 現地時間11月16日午後5時15分（日本時間17日午前10時15分）から約15分間、APEC首脳会議に出席するため米国サンフランシスコを訪問中の岸田文雄内閣 ... Apr 12, 2024 ... 岸田総理が米国から国賓待遇で招待を受け2024年4月8日～14日の予定で訪米している。4月10日にバイデン大統領との公式な日米首脳会談が行われ、会談後 ... Nov 15, 2023 ... APEC（アジア太平洋経済協力、Asia Pacific Economic Cooperation） · 岸田総理大臣の米国訪問（令和5年11月15日～19日） · APEC首脳会議 · IPEF首脳会合. Nov 17, 2023 ... 【NHK】岸田総理大臣は、訪問先のアメリカでバイデン大統領との首脳会談に臨み、中国への対応について、16日の米中首脳会談の結果など… Jan 13, 2023 ... を訪問中の岸田文雄内閣総理大臣は、ジョセフ・バイデン米国大統領（The Honorable Joseph R. Biden, Jr., President of the United States of America） ... Apr 11, 2024 ... 【NHK】アメリカを訪れている岸田総理大臣はバイデン大統領と会談し、自衛隊とアメリカ軍の指揮・統制の向上など、防衛協力を深めるとと… Feb 8, 2024 ... 岸田文雄首相は9日、政府与党連絡会議であいさつし、諸般の事情が許せば、国賓待遇で米国を公式に訪問し4月10日にワシントンⅮCで日米首脳会談を ... Apr 11, 2024 ... ... 日米両国がグローバルなパートナーとして真価を発揮すべき時だ」と述べた。 岸田首相は首脳会談前日の4月9日に米国商工会議所と米日経済協議会（USJBC ... 20

{'input': '岸田総理の日米首脳会談の結果について教えてください',
 'output': '岸田総理は2024年4月10日にバイデン大統領と公式な日米首脳会談を行いました。会談後、共同声明とファクトシートが出されました。会談では、日米両国がグローバルなパートナーとして真価を発揮すべきとの考えが述べられました。'}

# 対話型Google検索Agent

https://python.langchain.com/docs/modules/memory/agent_with_memory

In [17]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI

In [18]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="現在の出来事に関する質問に答える必要がある場合に便利です",
    )
]

In [19]:
prefix = """あなたは日本語で回答します。あなたは人間と会話し、次の質問にできる限り答えてください。次のツールにアクセスできます:"""

suffix = """開始！"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [20]:
llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [21]:
agent_chain.invoke({"input":"日本の人口は何人ですか？"})



> Entering new AgentExecutor chain...
Thought: I don't know the exact population of Japan
Action: Search
Action Input: "日本の人口は何人ですか"
Observation: 4 days ago ... ... 日本の人口 ... 統計調査等業務の業務・システム最適化と政府統計共同利用システム ... PDF形式のファイルをご覧いただく場合には、Adobe Readerが必要です。 Apr 17, 2024 ... ... です。 ＜目次＞ UNFPA事務局長の声明文：「世界人口白書2024」発表に寄せて 概要 主な調査結果 世界人口・日本人口（2024年） 関連資料・リンク ... Apr 12, 2023 ... 2022年（令和4年）10月1日現在の日本の人口について、年齢各歳別人口や都道府県別人口の推計結果を掲載しています。 May 30, 2022 ... 国連人口基金（UNFPA）が発表した「世界人口白書2022」によると、2022年の世界の総人口は79億5400万人だ。日本は11位で、前年と同じ順位だった。 Jan 22, 2024 ... 各月の「人口増減」は、当該1か月分の数値. 4 ... なお、最新推計月の3か月前と4か月前については ... 日本人人口 ＝基準人口（日本人）＋自然動態 ... Jun 13, 2023 ... 国連人口基金（UNFPA）の「世界人口白書2023」によると、2023年の世界人口は80億4500万人で初めて80億人に到達し、これまで1位だった中国をインドが ... 人口の多い国. 順位 ( じゅんい ), 国名, 推計 ( すいけい ) 人口 （1,000人）（2021年）. 1, 中国の国旗, 中華 ( ちゅうか ) ... 日本の国旗, 日本, 125,502. 注記 ( ちゅうき ). 約3億2000万人ですが、増加し続けています。U.S. and World Population. 世界ではアジアやアフリカの国々の人口が増えていく; 日本は世界のなかでも 高齢化 ( こうれいか ) が速く進んでいる. 日本の人口は約1億2600万人ですが、子どもの人口 ... 日本の将来推計人口. 表2

{'input': '日本の人口は何人ですか？', 'chat_history': '', 'output': '日本の人口は約1億2600万人です。'}

In [22]:
agent_chain.invoke(input="「その国の国歌を教えて")



> Entering new AgentExecutor chain...
Thought: I should search for the national anthem of Japan
Action: Search
Action Input: "National anthem of Japan"
Observation: May 8, 2010 ... "Kimigayo" (君が代?) is Japan's national anthem. It is also one of the world's shortest national anthems in current use, with a length of 11 ... "Kimigayo" (君が代, Japanese pronunciation: [kimiɡajo]; "His Imperial Majesty's Reign") is the national anthem of Japan. The lyrics are from a waka poem written ... Jul 31, 2022 ... Lyrics: Written during the Heian period (794–1185) Music: John William Fenton, 1869 Adopted: 1869, informally, 1999, made law. Jul 26, 2023 ... 'Kimigayo' is the country's single-verse national anthem, based on the words of a classical Japanese waka poem written by an unknown author ... The Rising Sun Flag and "Kimi Ga Yo" are respectively the national flag and anthem of Japan. This was formalized in 1999 with the Law regarding the National ... Jan 21, 2022 ... It came to existence in times 

{'input': '「その国の国歌を教えて',
 'chat_history': 'Human: 日本の人口は何人ですか？\nAI: 日本の人口は約1億2600万人です。',
 'output': '"Kimigayo" (君が代)'}

# Index検索Agent

In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# PDFの読込
loader = PyPDFLoader("./data2/001615363.pdf")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [24]:
from langchain.tools.retriever import create_retriever_tool

tools = [
  create_retriever_tool(
      retriever,
      "ForeignTravelersQASystem",
      "訪日外国人の旅行や土産品、消費動向に関する質問に役立ちます。",
  )
]

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2)

In [26]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
agent_executor.invoke({"input": "お土産の購入状況を教えて"})



> Entering new AgentExecutor chain...

Invoking: `ForeignTravelersQASystem` with `{'query': 'お土産の購入状況'}`


訪日外国人消費動向調査  
 
20 ３．土産品の購入実態  
 
（１） 費目別購入率 
 費目別の 購入率（その費目を購入した人の
割合）は「菓子類」 （ 70.4％）、「その他食
料品・飲料・たばこ」 （ 43.4％）、 「衣類」
（33.8％）の順で高 い（図表3-1）。 
 国籍・地域別で 最も購入率が 高い費目は 、
韓国、台湾、香港 、中国で「菓子類」 （ それ
ぞれ82.9％、72.9％、63.3％、73.1％）、
米国で「その他食料品・飲料・たばこ」
（54.1％）となっている 。  
 
（２）費目別購入者単価  
 費目別の購入者単価 （その費目を購入し
た人における当該費目の 1人当たり平均
支出）は、「 宝石・貴金属 」が8.3万円
と最も高い（図表 3-1）。 
 国籍・地域別では 、台湾と香港の「宝
石・貴金属 」（それぞれ 19.9万円、
16.6万円）が他の国籍・地域に比べ高
い。
  図表3-1 買物代の費目別購入率および購入者単価  （主要国籍･地域別 、全目的）

訪日外国人消費動向調査 
 
38 
 （３）買物場所 
百貨店・デパート 
原則として百貨店協会加盟の店舗 
家電量販店 
PCやカメラ、電気製品を専門に販売する店舗 
ファッション専門店 
服・鞄・靴・アクセサリー・高級腕時計等
を専門に販売する店舗 
100 円ショップ 
店内の商品を原則として 1 点 100 日本円均
一で 販売する形態の小売店 
高速道路の SA・道の駅 
高速道路のサービスエリア（SA）やパーキ
ングエリア（PA）、国土交通省に登録され
ている「道の駅」 
観光地の土産店 
観光地内にある小売店 
宿泊施設 
旅館・ホテル等の館内に併設されている土産店 
スーパーマーケット 
高頻度に消費される食料品や日用品等をセル
フサービスで短時間に買えるようにした小売
店舗 
コンビニエンスストア 
年中無休で長時間の営業を行い、小規模な
店舗において主に食品、日用雑貨等多数の
品種を扱う形態の小売店 
ドラッグストア 
医

{'input': 'お土産の購入状況を教えて',
 'output': 'お土産の購入実態に関する情報をお伝えします。\n\n1. 費目別の購入率:\n- 費目別の購入率は、「菓子類」が70.4%、「その他食料品・飲料・たばこ」が43.4%、「衣類」が33.8%となっています。\n\n2. 国籍・地域別での購入率が高い費目:\n- 韓国、台湾、香港、中国では「菓子類」が最も購入率が高くなっており、米国では「その他食料品・飲料・たばこ」が高いです。\n\n3. 買物場所:\n- 土産品の購入場所としては、百貨店・デパート、家電量販店、ファッション専門店、100円ショップ、高速道路のSA・道の駅、観光地の土産店、宿泊施設、スーパーマーケット、コンビニエンスストア、ドラッグストア、ディスカウントストアなどがあります。\n\n以上がお土産の購入実態に関する情報です。ご参考までにどうぞ。'}

# OpenAI Tool

In [30]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [31]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [32]:
import requests
from bs4 import BeautifulSoup
from langchain.chains import TransformChain

# ニュースを検索する関数
def get_news(inputs: dict) -> dict:
    keyword = inputs["keyword"]

    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return {"result": news}

# 関数をChain化
news_chain = TransformChain(
    input_variables=["keyword"],
    output_variables=["result"],
    transform=get_news
)


In [33]:
# 数式の文字列をもとに計算を行う関数
def calc_func(inputs: dict) -> dict:
    formula = inputs["formula"]
    answer = eval(formula)
    return {"answer": f"{answer}"}

# 関数をChain化
calc_chain = TransformChain(
    input_variables=["formula"],
    output_variables=["answer"],
    transform=calc_func
)

In [34]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "news", # 半角英数字のみ
        func=news_chain.run,
        description="ニュースを探す際に役立ちます"
    ),
    Tool(
        name = "calculation", # 半角英数字のみ
        func=calc_chain.run,
        description="計算をする場合に役立ちます"
    ),
    Tool(
        name = "Search", # 半角英数字のみ
        func=search.run,
        description="現在の出来事や世界の現状に関する質問に答える必要がある場合に役立ちます"
    ),
]

In [35]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [36]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
agent_executor.invoke({"input": "LangChainとは何ですか?"})



> Entering new AgentExecutor chain...

Invoking: `Search` with `LangChain`


From startups to global enterprises, ambitious builders choose LangChain. Build. LangChain gives developers a framework to construct LLM‑powered apps easily. What is LangChain? · Open-source libraries: Build your applications using LangChain's modular building blocks and components. · Productionization: Inspect, ... LangChain is a framework for developing applications powered by large language models (LLMs). Introduction. LangChain is a framework for developing applications powered by language models. It enables applications that: ... This framework consists of ... Quickstart · Get setup with LangChain, LangSmith and LangServe · Use the most basic and common components of LangChain: prompt templates, models, and output ... langchain.chains ¶. Chains are easily reusable components linked together. Chains encode a sequence of calls to components like models, document retrievers, ... May 8, 2023 ... LangChain i

{'input': 'LangChainとは何ですか?',
 'output': 'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、言語モデルによって動作するアプリケーションを開発するためのフレームワークであり、開発者に簡単にLLMを活用したアプリを構築するためのフレームワークを提供します。LangChainはオープンソースライブラリを使用してアプリケーションを構築し、製品化を行うことができます。LangChainは、PythonとJavaScriptで書かれており、AI開発者に言語モデルと外部データソースを結びつけるためのツールを提供しています。LangChainは、大規模言語モデル（LLM）を使用したアプリケーションの開発をサポートするオープンソースのオーケストレーションフレームワークです。'}

In [38]:
agent_executor.invoke({"input": "8掛ける12は？"})



> Entering new AgentExecutor chain...

Invoking: `calculation` with `8*12`


968掛ける12は96です。

> Finished chain.


{'input': '8掛ける12は？', 'output': '8掛ける12は96です。'}

In [39]:
agent_executor.invoke({"input": "夏祭りに関するニュースを教えてください"})



> Entering new AgentExecutor chain...

Invoking: `news` with `夏祭り`


[{'title': '【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...'}, {'title': 'THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days'}, {'title': '今年の夏は、ホテルで夏祭り！デザートビュッフェ「Bon Dance」を6月22日（土）から開催'}]最近の夏祭りに関するニュースです：
1. 【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...
2. THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days
3. 今年の夏は、ホテルで夏祭り！デザートビュッフェ「Bon Dance」を6月22日（土）から開催

どれか詳細を知りたい場合はお知らせください。

> Finished chain.


{'input': '夏祭りに関するニュースを教えてください',
 'output': '最近の夏祭りに関するニュースです：\n1. 【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...\n2. THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days\n3. 今年の夏は、ホテルで夏祭り！デザートビュッフェ「Bon Dance」を6月22日（土）から開催\n\nどれか詳細を知りたい場合はお知らせください。'}

# Toolkit:Pandas DataFrame Agent

In [40]:
import pandas as pd

df = pd.read_csv("./data2/iris.data", header=0)
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [41]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [42]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [44]:
agent.invoke("何行ありますか？")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


150データフレームには150行あります。

> Finished chain.


{'input': '何行ありますか？', 'output': 'データフレームには150行あります。'}

In [45]:
agent.invoke("Nameの基本統計量を出力してください")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Name'].describe()"}`


count             150
unique              3
top       Iris-setosa
freq               50
Name: Name, dtype: object`Name`列の基本統計量は以下の通りです:
- データ数: 150
- ユニークな値の数: 3
- 最も頻繁に現れる値: Iris-setosa (出現回数: 50)

> Finished chain.


{'input': 'Nameの基本統計量を出力してください',
 'output': '`Name`列の基本統計量は以下の通りです:\n- データ数: 150\n- ユニークな値の数: 3\n- 最も頻繁に現れる値: Iris-setosa (出現回数: 50)'}

In [46]:
agent.invoke("このデータについて説明してください")



> Entering new AgentExecutor chain...
このデータは、アヤメの花のがく片（Sepal）と花弁（Petal）の長さと幅を示しています。各行は1つのアヤメの花を表し、それぞれの花にはがく片の長さ（SepalLength）、がく片の幅（SepalWidth）、花弁の長さ（PetalLength）、花弁の幅（PetalWidth）が記録されています。また、各花の種類（Name）も示されており、ここでは"Iris-setosa"という種類が示されています。

> Finished chain.


{'input': 'このデータについて説明してください',
 'output': 'このデータは、アヤメの花のがく片（Sepal）と花弁（Petal）の長さと幅を示しています。各行は1つのアヤメの花を表し、それぞれの花にはがく片の長さ（SepalLength）、がく片の幅（SepalWidth）、花弁の長さ（PetalLength）、花弁の幅（PetalWidth）が記録されています。また、各花の種類（Name）も示されており、ここでは"Iris-setosa"という種類が示されています。'}

# Toolkit:Python Agent

In [48]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [49]:
tools = [PythonREPLTool()]

In [50]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [51]:
instructions = """あなたは、質問に答えるために Python コードを作成して実行するように設計されたエージェントです。
Python コードを実行するために使用できる Python REPL にアクセスできます。
エラーが発生した場合は、コードをデバッグして再試行してください。
コードの出力は質問に答えるためにのみ使用してください。
コードを実行しなくても答えがわかる場合もありますが、答えを得るにはコードを実行する必要があります。
質問に答えるコードを書けそうにない場合は、「わかりません」と答えてください。
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [52]:
agent = create_openai_functions_agent(ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0), tools, prompt)

In [53]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
agent_executor.invoke({"input": "100までの素数を求めよ"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(2, 101) if is_prime(num)]
primes`


100までの素数は、[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97] です。

> Finished chain.


{'input': '100までの素数を求めよ',
 'output': '100までの素数は、[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97] です。'}